# **Data Mining - MidTerm Project**

**Name**     : Aniket Khalate<br>
**UCID**     : ak3274<br>
**Email_id** : ak3274@njit.edu<br>

**A Comparative Analysis of Brute-Force, Apriori, and FP-Growth Algorithms**

**Introduction:**<br>
This project outlines the implementation of three algorithms; Brute-Force, Apriori, and FP-Growth for extracting frequent itemsets and generating association rules from transactional datasets. The objective is to evaluate the performance and efficiency of these algorithms in identifying frequent itemsets and rules, while analyzing their execution times across different support and confidence thresholds.

**Import Statements**<br>

This section imports the required libraries:

1. os – Provides functions for interacting with the operating system.

2. itertools – Contains tools for creating efficient iterators, including combinations and permutations.

3. time – Used for measuring execution durations and working with timestamps.

4. subprocess – Allows you to spawn new processes, run external commands, and interact with their input/output streams.

5. sys – Gives access to system-specific parameters and functions.

6. importlib.util - it provides utility functions for working with modules.

7. mlxtend.frequent_patterns - it provies apriori, fpgrowth, and association_rules for implementing the Apriori and FP-Growth algorithms.

8. pandas - for data manipulation.

In [1]:
import os
import subprocess
import sys
import itertools
import time
import importlib.util
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

**Support Calculation**<br>

This function calculates the support of a given itemset which is nothing but the proportion of transactions that contain all the items in the itemset.

In [2]:
def compute_support(transaction, itemset):
    return sum(1 for t in transaction if set(itemset).issubset(set(t))) / len(transaction)

**Brute Force Frequent Itemset Generation**<br>

This function uses a brute force approach to find all frequent itemsets. It systematically generates all possible item combinations of increasing size, calculates their support, and keeps the ones that meet the minimum support threshold.

In [3]:
def brute_force_frequent_itemsets(transaction, min_support):
    unique_items = sorted(set(item for t in transaction for item in t))  # all unique items
    all_frequent_itemsets = []
    k = 1

    while True:
        candidate_sets = list(itertools.combinations(unique_items, k))
        frequent_candidates = []

        for candidate in candidate_sets:
            support = compute_support(transaction, candidate)
            if support >= min_support:
                frequent_candidates.append((candidate, support))

        if not frequent_candidates:
            break

        all_frequent_itemsets.extend(frequent_candidates)
        k += 1

    return all_frequent_itemsets

**Association Rule Generation**<br>

This function derives association rules from the frequent itemsets. For each itemset, it splits items into possible antecedent consequent pairs, calculates the confidence of each rule, and keeps the ones that meet the minimum confidence threshold.

In [4]:
def generate_association_rules(frequent_itemsets, transaction, min_confidence):
    association_rules = []

    for itemset, itemset_support in frequent_itemsets:
        if len(itemset) > 1:
            for r in range(1, len(itemset)):
                for antecedent_items in itertools.combinations(itemset, r):
                    consequent_items = tuple(item for item in itemset if item not in antecedent_items)
                    antecedent_support = compute_support(transaction, antecedent_items)
                    confidence = itemset_support / antecedent_support
                    if confidence >= min_confidence:
                        association_rules.append((antecedent_items, consequent_items, confidence, itemset_support))

    return association_rules

**Transaction DataFrame Preparation**<br>

This function converts the list of transactions into a boolean DataFrame, where each column represents an item and each row represents a transaction. A cell is True if the item is present in that transaction, and False otherwise.

In [5]:
def prepare_transaction_df(transaction):
    unique_items = sorted(set(item for t in transaction for item in t))
    df = pd.DataFrame(
        [[item in t for item in unique_items] for t in transaction],
        columns=unique_items
    ).astype(bool)
    return df

**Algorithm Execution and Comparison**<br>

This function runs all three algorithms; Brute Force, Apriori, and FP-Growth on the given transaction data. It measures their execution times, generates frequent itemsets and association rules for each, prints the results, and returns the time taken by each algorithm for performance comparison.

In [6]:
def run_all_algorithms(transaction, min_support, min_confidence):

    transaction_df = prepare_transaction_df(transaction)

    #  Brute Force  #
    print("\nRunning Brute Force Algorithm...")
    start_time = time.time()
    frequent_itemsets_brute = brute_force_frequent_itemsets(transaction, min_support)
    rules_brute = generate_association_rules(frequent_itemsets_brute, transaction, min_confidence)
    brute_time = time.time() - start_time
    print(f"Brute Force Time: {brute_time:.4f} seconds")
    print_results("Brute Force", frequent_itemsets_brute, rules_brute)

    #  Apriori  #
    print("\nRunning Apriori Algorithm...")
    start_time = time.time()
    try:
        frequent_itemsets_apriori = apriori(transaction_df, min_support=min_support, use_colnames=True)
        apriori_time = time.time() - start_time

        if frequent_itemsets_apriori.empty:
            print("Apriori did not find any frequent itemsets.")
        else:
            rules_apriori = association_rules(
                frequent_itemsets_apriori, metric="confidence", min_threshold=min_confidence
            )
            print(f"Apriori Time: {apriori_time:.4f} seconds")
            print_results("Apriori", frequent_itemsets_apriori, rules_apriori)

    except Exception as e:
        apriori_time = time.time() - start_time
        print(f"An error occurred during Apriori algorithm execution: {e}")

    # FP-Growth #
    print("\nRunning FP-Growth Algorithm...")
    start_time = time.time()
    try:
        frequent_itemsets_fpgrowth = fpgrowth(transaction_df, min_support=min_support, use_colnames=True)
        fpgrowth_time = time.time() - start_time

        if frequent_itemsets_fpgrowth.empty:
            print("FP-Growth did not find any frequent itemsets.")
        else:
            rules_fpgrowth = association_rules(
                frequent_itemsets_fpgrowth, metric="confidence", min_threshold=min_confidence
            )
            print(f"FP-Growth Time: {fpgrowth_time:.4f} seconds")
            print_results("FP-Growth", frequent_itemsets_fpgrowth, rules_fpgrowth)

    except Exception as e:
        fpgrowth_time = time.time() - start_time
        print(f"An error occurred during FP-Growth algorithm execution: {e}")


    return brute_time, apriori_time, fpgrowth_time

**CSV Reading and Transaction Preparation**<br>

This function reads transaction and item data from CSV files, optionally maps item numbers to item names, and processes each row into a clean list of transactions that can be used for frequent itemset mining.

In [7]:
def read_csv_and_prepare_transactions(transaction_file, itemset_file):
    try:
        df_transactions = pd.read_csv(transaction_file)
        df_items = pd.read_csv(itemset_file)

        # Create a mapping of item numbers to item names if available
        if 'Item #' in df_items.columns and 'Item Name' in df_items.columns:
            item_mapping = dict(zip(df_items['Item #'], df_items['Item Name']))
        else:
            item_mapping = None

        transaction_list = []
        for _, row in df_transactions.iterrows():
            transaction = []
            for entry in row:
                if isinstance(entry, str):
                    items = [i.strip() for i in entry.split(',') if i.strip()]
                    transaction.extend(items)
                elif pd.notna(entry):
                    transaction.append(str(entry))
            if transaction:
                transaction_list.append(transaction)

        return transaction_list

    except Exception as e:
        print(f"Error reading the CSV files: {str(e)}")
        raise

**Result Printing**<br>

This function displays the frequent itemsets and association rules generated by different algorithms. It handles both DataFrame and list formats, printing itemsets with their support values and rules with their confidence and support in a readable format.

In [8]:
def print_results(algorithm_name, frequent_itemsets, rules):
    print(f"\n{algorithm_name} Results:")

    # Print Frequent Itemsets
    print("Frequent Itemsets:")
    if isinstance(frequent_itemsets, pd.DataFrame):
        for _, row in frequent_itemsets.iterrows():
            items = set(row['itemsets'])
            support = row['support']
            print(f"Items: {items}, Support: {support*100:.2f}%")
    else:
        for itemset, support in frequent_itemsets:
            items = set(itemset)
            print(f"Items: {items}, Support: {support*100:.2f}%")

    # Print Association Rules
    print("\nAssociation Rules:")
    if isinstance(rules, pd.DataFrame):
        for _, rule in rules.iterrows():
            antecedent_items = set(rule['antecedents'])
            consequent_items = set(rule['consequents'])
            confidence = rule['confidence']
            support = rule['support']
            print(f"Rule: {antecedent_items} -> {consequent_items}")
            print(f"Confidence: {confidence*100:.2f}%, Support: {support*100:.2f}%\n")
    else:
        for antecedent, consequent, confidence, support in rules:
            antecedent_items = set(antecedent)
            consequent_items = set(consequent)
            print(f"Rule: {antecedent_items} -> {consequent_items}")
            print(f"Confidence: {confidence*100:.2f}%, Support: {support*100:.2f}%\n")

**Main Program Execution:**<br>

This function serves as the main workflow for the script. It lets the user select a store, reads and prepares the corresponding transaction data, prompts for minimum support and confidence values, runs all three algorithms, prints the results and execution times, and identifies the fastest algorithm.

In [9]:
def main():

    # Get the folder where this script is located
    base_dir = os.getcwd()


    # Define available stores and their corresponding files
    stores = {
    "1": ("Amazon",
          os.path.join(base_dir, "Amazon_Transaction.csv"),
          os.path.join(base_dir, "Amazon_Itemset.csv")),
    "2": ("Bestbuy",
          os.path.join(base_dir, "Bestbuy_Transaction.csv"),
          os.path.join(base_dir, "Bestbuy_Itemset.csv")),
    "3": ("General",
          os.path.join(base_dir, "General_Transaction.csv"),
          os.path.join(base_dir, "General_Itemset.csv")),
    "4": ("K-mart",
          os.path.join(base_dir, "K_mart_Transaction.csv"),
          os.path.join(base_dir, "K_mart_Itemset.csv")),
    "5": ("Nike",
          os.path.join(base_dir, "Nike_Transaction.csv"),
          os.path.join(base_dir, "Nike_Itemset.csv"))
}


    print("\nAvailable stores:")
    for key, (store_name, _, _) in stores.items():
        print(f"{key}. {store_name}")

    # Get user input for store selection.
    while True:
        selected_store = input("Enter the number of the store you want to analyze: ")
        if selected_store in stores:
            store_label, trans_file, item_file = stores[selected_store]
            break
    else:
        print("Invalid choice. Please enter a number between 1 and 5.")


    try:
        transaction_list = read_csv_and_prepare_transactions(trans_file, item_file)
        if not transaction_list:
            print("No valid transactions found. Please check your CSV files.")
            return
    except Exception as e:
        print(f"Error reading the CSV files: {e}")
        print("Please ensure that the CSV files are properly formatted.")
        return


# Input for minimum support and minimum confidence.
    while True:
        try:
            min_supp = float(input("Enter the minimum support (as a percentage between 0 and 100): "))
            if 0 <= min_supp <= 100:
                min_supp /= 100
                break
            else:
                print("Please enter a value between 0 and 100.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    while True:
        try:
            min_conf = float(input("Enter the minimum confidence (as a percentage between 0 and 100): "))
            if 0 <= min_conf <= 100:
                min_conf /= 100
                break
            else:
                print("Please enter a value between 0 and 100.")
        except ValueError:
            print("Invalid input. Please enter a number.")


    brute_time, apriori_time, fpgrowth_time = run_all_algorithms(transaction_list, min_supp, min_conf)

    # Print execution times
    print(f"\nExecution Times:")
    print(f"Brute Force: {brute_time:.4f} seconds")
    print(f"Apriori: {apriori_time:.4f} seconds")
    print(f"FP-Growth: {fpgrowth_time:.4f} seconds")

    # Identify the fastest algorithm
    fastest_algorithm = min(
        (brute_time, 'Brute Force'),
        (apriori_time, 'Apriori'),
        (fpgrowth_time, 'FP-Growth')
    )[1]
    print(f"\nThe fastest algorithm is: {fastest_algorithm}")


if __name__ == "__main__":
    main()


Available stores:
1. Amazon
2. Bestbuy
3. General
4. K-mart
5. Nike

Running Brute Force Algorithm...
Brute Force Time: 0.0524 seconds

Brute Force Results:
Frequent Itemsets:
Items: {'A'}, Support: 91.67%
Items: {'B'}, Support: 41.67%
Items: {'C'}, Support: 58.33%
Items: {'D'}, Support: 33.33%
Items: {'E'}, Support: 66.67%
Items: {'B', 'A'}, Support: 41.67%
Items: {'A', 'C'}, Support: 58.33%
Items: {'A', 'D'}, Support: 33.33%
Items: {'A', 'E'}, Support: 66.67%
Items: {'B', 'C'}, Support: 33.33%
Items: {'B', 'D'}, Support: 25.00%
Items: {'B', 'E'}, Support: 16.67%
Items: {'D', 'C'}, Support: 16.67%
Items: {'E', 'C'}, Support: 33.33%
Items: {'D', 'E'}, Support: 25.00%
Items: {'B', 'A', 'C'}, Support: 33.33%
Items: {'B', 'A', 'D'}, Support: 25.00%
Items: {'B', 'A', 'E'}, Support: 16.67%
Items: {'A', 'D', 'C'}, Support: 16.67%
Items: {'A', 'E', 'C'}, Support: 33.33%
Items: {'A', 'D', 'E'}, Support: 25.00%
Items: {'B', 'D', 'C'}, Support: 16.67%
Items: {'B', 'D', 'E'}, Support: 16.67%
Ite